In [47]:
from datetime import date, datetime

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, select, update, event, Date, DateTime, exists, and_, UniqueConstraint, ForeignKeyConstraint
from sqlalchemy.orm import Session, declarative_base, relationship

engine = create_engine("sqlite:///:memory:", echo=True, future=True)

def _fk_pragma_on_connect(dbapi_con, con_record):
    dbapi_con.execute('PRAGMA foreign_keys = 1')

if engine.url.drivername == 'sqlite':
    event.listen(engine, 'connect', _fk_pragma_on_connect)
    print("Added FOREIGN_KEY pragma event for sqlite.")

Base = declarative_base()


class User(Base):
    __tablename__ = 'user_accounts'

    name = Column(String(30), primary_key=True)
    fullname = Column(String)
   
    def __eq__(self, other):
        return self.name == other.name

    def __repr__(self):
       return f"User(name={self.name}, fullname={self.fullname})"


Base.metadata.create_all(engine)

sandy = User(name="sandy", fullname="Sandy Cheeks")
new_sandy = User(name="sandy", fullname="Sandy New")
with Session(engine) as session:
    session.add(sandy)
    session.commit()
    print(session.execute(select(User)).all())

Added FOREIGN_KEY pragma event for sqlite.
2022-03-20 01:59:14,404 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-20 01:59:14,406 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_accounts")
2022-03-20 01:59:14,407 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-20 01:59:14,409 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_accounts")
2022-03-20 01:59:14,409 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-20 01:59:14,413 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_accounts (
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (name)
)


2022-03-20 01:59:14,414 INFO sqlalchemy.engine.Engine [no key 0.00079s] ()
2022-03-20 01:59:14,417 INFO sqlalchemy.engine.Engine COMMIT
2022-03-20 01:59:14,423 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-20 01:59:14,427 INFO sqlalchemy.engine.Engine INSERT INTO user_accounts (name, fullname) VALUES (?, ?)
2022-03-20 01:59:14,428 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('sandy', '

In [44]:
session = Session(engine)
session.add(new_sandy)
print(sandy in session)
print(new_sandy in session)

# session.commit()
session.close()

False
True


In [43]:
session = Session(engine)
users = session.execute(select(User)).scalars().all()

print(sandy in session)
print(new_sandy in session)

print(sandy in users)
print(new_sandy in users)

session.close()

2022-03-17 15:48:17,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-17 15:48:17,312 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname 
FROM user_accounts
2022-03-17 15:48:17,313 INFO sqlalchemy.engine.Engine [cached since 2083s ago] ()
False
False
True
True
2022-03-17 15:48:17,315 INFO sqlalchemy.engine.Engine ROLLBACK


In [ ]:
# adding to a session does not check for primary key uniquity violation, but the database itself does, when committing.
# querying instances does not add them to the session. (????)
# session contains references, so when checking for containing instances with 'in', actual references are checked, not __eq__ or primary_key
# query result contains instances, so when checking for containing instances with 'in', instances are checked by usage of __eq__

In [46]:
session = Session(engine)

session.add(new_sandy)
print(session.identity_map.values())

session.close()

[]


In [7]:
with Session(engine) as session:
    existing_users = session.execute(select(User)).scalars().all()
    print(existing_users)
    if new_sandy in existing_users:
        user_to_update = session.execute(select(User).where(User.name == new_sandy.name)).scalar_one()
        user_to_update.fullname = new_sandy.fullname
    session.commit()

2022-03-10 17:48:05,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-10 17:48:05,540 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname 
FROM user_accounts
2022-03-10 17:48:05,540 INFO sqlalchemy.engine.Engine [cached since 359.4s ago] ()
[User(name=sandy, fullname=Sandy New)]
2022-03-10 17:48:05,543 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname 
FROM user_accounts 
WHERE user_accounts.name = ?
2022-03-10 17:48:05,543 INFO sqlalchemy.engine.Engine [cached since 19.4s ago] ('sandy',)
2022-03-10 17:48:05,545 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
with Session(engine) as session:
    print(session.execute(select(User)).scalars().all())

2022-03-10 17:48:32,297 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-10 17:48:32,298 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname 
FROM user_accounts
2022-03-10 17:48:32,299 INFO sqlalchemy.engine.Engine [cached since 386.1s ago] ()
[User(name=sandy, fullname=Sandy New)]
2022-03-10 17:48:32,300 INFO sqlalchemy.engine.Engine ROLLBACK


In [54]:
with Session(engine) as session:
    all_users = session.execute(select(User)).scalars().all()
    for user in users:
        print(user)
    another_sandy = User(name="sandy", fullname="Sandy New")
    session.flush()
    session.add(another_sandy)
    session.commit()
    # DOES THIS CHANGE WHEN USING POSTGRES????
    # BESTEHEN DIE CONTRACTS IN DER IDENTITIY MAP?
    # PROBLEM ISOLIERT NACHSTELLEN

2022-03-20 02:03:22,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-20 02:03:22,895 INFO sqlalchemy.engine.Engine SELECT user_accounts.name, user_accounts.fullname 
FROM user_accounts
2022-03-20 02:03:22,895 INFO sqlalchemy.engine.Engine [cached since 248.5s ago] ()
User(name=sandy, fullname=Sandy Cheeks)
2022-03-20 02:03:22,900 INFO sqlalchemy.engine.Engine INSERT INTO user_accounts (name, fullname) VALUES (?, ?)
2022-03-20 02:03:22,900 INFO sqlalchemy.engine.Engine [cached since 248.5s ago] ('sandy', 'Sandy New')
2022-03-20 02:03:22,902 INFO sqlalchemy.engine.Engine ROLLBACK


/var/folders/qt/k5txb7556tz0pk9dmksrt2sr0000gn/T/ipykernel_87545/989246536.py:8: SAWarning: New instance <User at 0x112ebce80> with identity key (<class '__main__.User'>, ('sandy',), None) conflicts with persistent instance <User at 0x11308c4c0>
  session.commit()


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: user_accounts.name
[SQL: INSERT INTO user_accounts (name, fullname) VALUES (?, ?)]
[parameters: ('sandy', 'Sandy New')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)